In [55]:
import torch
import triton
import triton.language as tl

In [56]:
@triton.jit
def add_kernel(
    x_ptr, y_ptr,
    p, seed, 
    buffer_ptr,
    output_ptr,
    n_elements,
    BLOCK_SIZE: tl.constexpr,
):
    pid = tl.program_id(0)
    offs = pid * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
    mask = offs < n_elements
    x_ptrs = x_ptr + offs
    y_ptrs = y_ptr + offs
    x = tl.load(x_ptrs, mask=mask)
    y = tl.load(y_ptrs, mask=mask)
    z = x + y
    output_ptrs = output_ptr + offs
    rand = tl.zeros((BLOCK_SIZE,), dtype=tl.float32)
    rand = tl.rand(seed, tl.arange(0, BLOCK_SIZE))
    buffer_ptrs = buffer_ptr + offs
    tl.store(buffer_ptrs, rand, mask=mask)
    rand = rand>p
    tl.store(output_ptrs, rand, mask=mask)



In [57]:
def add(x, y, p, seed):
    output = torch.empty_like(x)
    buffer = torch.empty_like(x)
    assert x.is_cuda and y.is_cuda and output.is_cuda
    n_elements = output.numel()
    grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']), )
    add_kernel[grid](x, y, 
                     p, seed,
                     buffer,
                     output, n_elements, BLOCK_SIZE=4)
    print(buffer)
    print(output)
    return output

In [66]:
size = 4
x = torch.zeros(size, device='cuda')+2
y = torch.zeros(size, device='cuda')+1
print(x)
print(y)
# I don't understand
add(x,y, p=0.5, seed=1)

tensor([2., 2., 2., 2.], device='cuda:0')
tensor([1., 1., 1., 1.], device='cuda:0')


CompilationError: at 19:38:):
    pid = tl.program_id(0)
    offs = pid * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
    mask = offs < n_elements
    x_ptrs = x_ptr + offs
    y_ptrs = y_ptr + offs
    x = tl.load(x_ptrs, mask=mask)
    y = tl.load(y_ptrs, mask=mask)
    z = x + y
    output_ptrs = output_ptr + offs
    rand = tl.zeros((BLOCK_SIZE,), dtype=tl.float32)
    rand = tl.rand(seed, tl.arange(0, BLOCK_SIZE))
                                      ^
AttributeError("'constexpr' object has no attribute 'to'")